In [2]:
import pandas as pd
import numpy as np

from descriptiveAnalysis import *
from cropData import *

## Preparing Tariff Model Data

prepare(data) Creates the dataset for the tariff prediction model to be passed into, containing our input feautures as well as our output goal - mfn rate. The following input features currently included are listed below:


* `Crop ID`: The official ID of crops listed by the World Customs Organization.
* `FAO Cropname`: The registered name of each crop from the United Nations Food and Agriculture Organization.
* `Crop Group`: The crop groups for each listed crop from the UN FAO.
* `Country`: The country the product is being imported to.
* `Iso3Code`: The 3-character identifier code for the importing country.
* `GDP Per Capita`: GDP per capita, in US dollars, of the importing country.
* `Percent GDP From Agriculture`: The percent that the agriculture industry of importing country contributes to annual GDP.
* `Democracy Level`: The level of presence of electoral democracy of importing country, created by the V-Dem instute. Ranges from 0 to 1.
* `Production Quantity`: The total amount of the crop's crop group produced in a given eyar by importing country.
* `Import & Export Trade Value`: The total valuation of the crop groups imports and exports from importing country, taken from the United Nations Comtrade Database.

* Auxiliary Identifiers: the manner in which the crop product is prepared for shipping. Denoted with a `1` if crop contains property, and a `0` if not.
    - `Edible`
    - `Modified`
    - `Fresh`
    - `Frozen`
    - `Shelled`
    - `Dried`
    - `Chilled`
    - `Seed`
    - `Ground`
    - `Preserved`
    

In [3]:
def prepare_data(match_data, tariff_data, polyarchy_data, production_data, trade_data, gdp_industry_data):
    match_data = pd.read_csv(match_data)
    tariff_data = pd.read_csv(tariff_data)
    polyarchy_data = pd.read_csv(polyarchy_data)
    polyarchy_data = polyarchy_data.loc[polyarchy_data['year'] > 2020, ['country_name', 'country_text_id', 'v2x_polyarchy']]
    polyarchy_data.to_csv('../Output_Data/polyarchy_data.csv')

    production_data = pd.read_csv(production_data)
    trade_data = pd.read_csv(trade_data)
    gdp_industry_data = pd.read_csv(gdp_industry_data)
    data = {
        'Crop ID': [],
        'FAO Cropname': [],
        'Crop Group': [],
        'Country': [],
        'iso3code': [],
        'gdp per capita': [],
        'percent gdp from agriculture': [],
        'democracy level': [],
        'production quantity': [],
        'import trade value': [],
        'export trade value': [],
        'edible': [],
        'modified': [],
        'fresh': [],
        'frozen': [],
        'shelled': [],
        'dried': [],
        'chilled': [],
        'seed': [],
        'ground': [],
        'preserved': [],
        'mfn': []
    }

    gdp_dict = {}
    for index, row in gdp_industry_data.iterrows():
        if row['Country Name'] not in gdp_dict:
            gdp_dict[row['Country Name']] = [row['2018 [YR2018]']]
        else:
            gdp_dict[row['Country Name']].append(row['2018 [YR2018]'])

    for index, row in tariff_data.iterrows():
        if index % 17500 == 0:
            percent = round(index/700000,2)*100
            print(str(percent) + '%')
        if row['hs6'] in set(match_data['Crop ID']):
            for i, r in match_data.iterrows():
                if row['hs6'] == r['Crop ID']:
                    data['Crop ID'].append(r['Crop ID'])
                    data['FAO Cropname'].append(r['FAO Cropname'])
                    data['Crop Group'].append(r['Group'])
                    data['Country'].append(row['name'])
                    data['edible'].append(r['edible'])
                    data['modified'].append(r['modified'])
                    data['fresh'].append(r['fresh'])
                    data['frozen'].append(r['frozen'])
                    data['shelled'].append(r['shelled'])
                    data['dried'].append(r['dried'])
                    data['chilled'].append(r['chilled'])
                    data['seed'].append(r['seed'])
                    data['ground'].append(r['ground'])
                    data['preserved'].append(r['preserved'])
                    data['mfn'].append(row['mfn_rate'])

                    if r['FAO Cropname'] in set(production_data.loc[production_data['Area'] == row['name']]['Item']):
                        for i2, r2 in production_data.loc[production_data['Area'] == row['name']].iterrows():
                            if r2['Item'] == r['FAO Cropname']:
                                data['production quantity'].append(r2['Value'])
                                break
                    else:
                        data['production quantity'].append('NA')
                    break
            
            if row['iso3code'] in set(polyarchy_data['country_text_id']):
                for i, r in polyarchy_data.iterrows():
                    if row['iso3code'] == r['country_text_id']:
                        data['iso3code'].append(row['iso3code'])
                        data['democracy level'].append(r['v2x_polyarchy'])
                        break
            else:
                data['iso3code'].append('NA')
                data['democracy level'].append('NA')

            commodity_code = 0
            if int(str(row['hs6'])[0:2]) > 21:
                commodity_code = int(str(row['hs6'])[0])
            else:
                commodity_code = int(str(row['hs6'])[0:2])
            
            import_found = False
            export_found = False
            for i, r in trade_data.loc[trade_data['Reporter'] == row['name']].iterrows():
                if commodity_code == r['Commodity Code'] and row['name'] == r['Reporter'] and r['Trade Flow'] == 'Import':
                    data['import trade value'].append(r['Trade Value (US$)'])
                    import_found = True
                elif commodity_code == r['Commodity Code'] and row['name'] == r['Reporter'] and r['Trade Flow'] == 'Export':
                    data['export trade value'].append(r['Trade Value (US$)'])
                    export_found = True

                if import_found and export_found:
                    break
            if not import_found:
                data['import trade value'].append('NA')
            if not export_found:
                data['export trade value'].append('NA')

            if  row['name'] not in gdp_dict or gdp_dict[row['name']][0] == '..':
                data['percent gdp from agriculture'].append('NA')
            else:
                data['percent gdp from agriculture'].append(gdp_dict[row['name']][0])
            if row['name'] not in gdp_dict or gdp_dict[row['name']][1] == '..':
                data['gdp per capita'].append('NA')
            else:
                data['gdp per capita'].append(gdp_dict[row['name']][1])

    output = pd.DataFrame.from_dict(data)

    output.to_csv('../Output_Data/tariff_model_data.csv')

In [ ]:
print('Building Data...')
prepare_data('../Output_Data/tariff_prod_match_merge_hs6_2017.csv', 'wits_tariff_2018.csv', 'Country_Year_V-Dem_Core_CSV_v12/V-Dem-CY-Core-v12.csv', 'FAOSTAT_data_prod_quant_2018.csv', 'comtrade_2018.csv', 'gdp_and_industry.csv')


## Comparing Crop Auxiliary Properties

If we would like to compare a crop by its different auxiliary identifier, we can do so with the function below, which outputs the difference in average tariff rates of a crop between two auxiliary identifiers for every country.

The inputs are as follows:

* `tariff_data`: the tariff dataset constructed above.
* `category`: the crop type the function filters by in collecting the average tariff rate.
* `auxiliary[1,2]`: the 2 auxiliary types of the crop group we are comparing

In [4]:
def compare_crop_properties(tariff_data, category, auxiliary1, auxiliary2):
    output = {}
    boxplot = {}
    tariff_data = pd.read_csv(tariff_data)
    for index, val in enumerate(category):

        mfn_dict = {}
        for i, row in tariff_data.iterrows():
            if str(row['FAO Cropname']) == 'nan' or row['Country'] != 'Turkey':
                continue

            label1 = row['Country'] + ' ' + auxiliary1[index]
            label2 = row['Country'] + ' ' + auxiliary2[index]
            
            if (val in row['FAO Cropname'] or val in row['Crop Group']) and row[auxiliary1[index]] == 1:
                if label1 not in mfn_dict:
                    mfn_dict[label1] = []
                mfn_dict[label1].append(row['mfn'])

            if (val in row['FAO Cropname'] or val in row['Crop Group']) and row[auxiliary2[index]] == 1:
                if label2 not in mfn_dict:
                    mfn_dict[label2] = []
                mfn_dict[label2].append(row['mfn'])

        '''
        df = {
            'Country': [],
            avg1: [],
            avg2: [],
        }
        
        for country in mfn_dict:
            df['Country'].append(country)
            df['Set size'].append(len(mfn_dict[country]))
            df[avg1].append(round(sum(mfn_dict[country])/len(mfn_dict[country]),2))
            df[avg2].append(round(sum(mfn_dict[country])/len(mfn_dict[country]),2))
        '''
        for country_crop in mfn_dict:
            mfn_dict[country_crop] = round(sum( mfn_dict[country_crop])/len( mfn_dict[country_crop]),2)

        label = auxiliary1[index] + ' vs ' + auxiliary2[index] + ' ' + val
        output[label] = {}
        boxplot[label] = []
        for country in list(set(tariff_data['Country'])):
            label1 = country + ' ' + auxiliary1[index]
            label2 = country + ' ' + auxiliary2[index]
            if label1 in mfn_dict and label2 in mfn_dict:
                output[label][country]= mfn_dict[label1] - mfn_dict[label2]
                boxplot[label].append(mfn_dict[label1] - mfn_dict[label2])
        boxplot[label] = sorted(boxplot[label])
    #print(mfn_dict)
    for prop in output:
        print(prop)
        for country in output[prop]:
            if abs(output[prop][country]) > 1:
                print(country + ': ' + str(output[prop][country]))

    for crop in boxplot:
        print(crop, boxplot[crop])
    #output = pd.DataFrame.from_dict(df)
    #output.to_csv('../Output_Data/filtered_country_data.csv')

In [ ]:
# crop properties - ['berries', 'Vegetables&Melons', 'Pulses', 'Oilcrop', 'Cereal', 'Treenuts'], ['preserved', 'chilled', 'preserved', 'modified', 'seed', 'shelled'], ['fresh', 'dried', 'shelled', 'frozen', 'ground', 'dried']

compare_crop_properties('../Output_Data/tariff_model_data.csv', ['berries', 'Vegetables&Melons', 'Pulses', 'Oilcrop', 'Cereal', 'Treenuts'], ['fresh']*6, ['frozen']*6)


# find_crop_rate

`find_crop_rate` returns the average mfn rate of a desired crop, which can be specified down to its auxiliary description and/or a chosen importing country or group of countries.

Inputs:
* `tariff_data`: data constructed in the functions above.
* `auxiliary`: desired auxiliary description. If `None`, this input is ignored.
* `country`: desired importing country, or list of countries. If `None`, this input is ignored.

In [8]:
def find_crop_rate(tariff_data, category, auxiliary=None, country = None):
    output = {}
    boxplot = {}
    tariff_data = pd.read_csv(tariff_data)
    for index, val in enumerate(category):
        mfn_dict = {}
        for i, row in tariff_data.iterrows():
            if str(row['FAO Cropname']) == 'nan':
                continue

            label = row['Country']
            
            if (val in row['FAO Cropname'] or val in row['Crop Group']) and (auxiliary == None or row[auxiliary[index]] == 1):
                if label not in mfn_dict:
                    mfn_dict[label] = []
                mfn_dict[label].append(row['mfn'])

        for country_crop in mfn_dict:
            mfn_dict[country_crop] = round(sum( mfn_dict[country_crop])/len( mfn_dict[country_crop]),2)

    if country == None:
        print(mfn_dict)
    else:
        print(auxiliary, category, country, mfn_dict[country])

In [9]:
find_crop_rate('../Output_Data/tariff_model_data.csv', ['Vegetables&Melons'], ['fresh'], 'Mexico')


['fresh'] ['Vegetables&Melons'] Mexico 11.37


# Statistical Modeling: Tariff & Democracy Histograms

We generated histrograms to plot the distribution of tariff rates (0-100%) and democracy levels (0-1) amongst tariff entires and countries.
    

In [ ]:
# Generate histogram model for mfn rates

df2 = pd.read_csv('../Raw_Data/wits_tariff_2018.csv')  
mfn_rates = list(df2['mfn_rate'])
mfn_rates.sort()

freq_dict = {}
for mfn in mfn_rates:
    if round(mfn,1) not in freq_dict:
        freq_dict[round(mfn,1)] = 1
    else:
        freq_dict[round(mfn,1)] += 1



age_bins = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
        8: 0,
        9: 0,
        10: 0,
        11: 0,
        12: 0,
        13: 0,
        14: 0,
        '15+': 0
    }

for index, val in enumerate(mfn_rates):
    age_bin = (val)//5
    if age_bin > 14:
        age_bins['15+'] += 1
    else:
        age_bins[age_bin] += 1
print('age bins:', age_bins)

In [ ]:
# Generate histogram model for mfn rates
tariff = pd.read_csv('../Raw_Data/wits_tariff_2018.csv')  
polyarchy = pd.read_csv('polyarchy_data.csv')  
polyarchy_bins = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
        6: [],
        7: [],
        8: [],
        9: [],
    }


bins = {
        '0.01-0.1': [],
        '0.11-0.2': [],
        '0.21-0.3': [],
        '0.31-0.4': [],
        '0.41-0.5': [],
        '0.51-0.6': [],
        '0.61-0.7': [],
        '0.71-0.8': [],
        '0.81-0.9': [],
        '0.91-1.0': [],
    }

p_dict = {}

for i, r in polyarchy.iterrows():
    p_dict[r['country_text_id']] = r['v2x_polyarchy']

for index, row in tariff.iterrows():
    country = row['iso3code']
    if index % 100000 == 0:
        print(index)
    mfn = row['mfn_rate']
    if country not in set(polyarchy['country_text_id']):
        continue
    elif country in p_dict:
        num = int(str(p_dict[country])[2])
        polyarchy_bins[num].append(mfn)
                
for mfn in polyarchy_bins:
    polyarchy_bins[mfn] = sum(polyarchy_bins[mfn])/len(polyarchy_bins[mfn])
    
print(polyarchy_bins)